In [1]:
import re
import pandas as pd
from tqdm.notebook import tqdm

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [2]:
# 事前学習済みの日本語感情分析モデルとそのトークナイザをロード
# model_name = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
model_name = 'Mizuiro-sakura/luke-japanese-large-sentiment-analysis-wrime'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

# 感情分析のためのパイプラインを設定
# nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, truncation=True)

# 使用するデバイスの設定
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# モデルをデバイスに転送
model = model.to(device)


In [3]:
pd_Φ  = pd.read_parquet('./parquet_data/20250227_tweet_EM_Φ.parquet')
words = [re.sub(r'単語\d+:', '', elem) for elem in pd_Φ.columns]
# sentiment_list = [label for label in model.config.id2label.values()]
sentiment_list = ['喜び', '悲しみ', '期待', '驚き', '怒り', '恐れ', '嫌悪', '信頼']
sentiment_list

['喜び', '悲しみ', '期待', '驚き', '怒り', '恐れ', '嫌悪', '信頼']

In [4]:
res_class  = []
batch_size = 64
for idx in range(0, len(words), batch_size):
	targets = words[idx:idx+batch_size]
    
	inputs  = tokenizer(targets, return_tensors='pt', padding=True, truncation=True, max_length=512)
	inputs  = {key: value.to(device) for key, value in inputs.items()}
	outputs = model(**inputs)
	probab  = torch.softmax(outputs.logits, dim=1).tolist()
 
	# 0:negative, 1:neutral, 2:positive
	res_class.extend(probab)

# 各単語の分類結果の保存
pd_sentiment = pd.DataFrame(data=[[a]+b for a,b in zip(words, res_class)], columns=['word'] + sentiment_list)
pd_sentiment.to_parquet('./parquet_data/20250227_tweet_sentiment.parquet')
    

In [5]:
# EMアルゴリズムによる感情分析の結果を保存
pd_Φ = pd.read_parquet('./parquet_data/20250227_tweet_EM_Φ.parquet')
for label in sentiment_list:
	pd_Φ[label] = 0.0
# sentiment_listの分だけpd_Φの列が増えているが、zip関数の挙動的に要素数の少ないres_classに合わせて動作するため問題ない
for col, res in zip(pd_Φ.columns, res_class):
    # 各単語の感情分析結果を重み付けして感情分析結果を算出
	for label in sentiment_list:
		pd_Φ.loc[:, label] = pd_Φ.loc[:, label] + pd_Φ.loc[:, col] * res[sentiment_list.index(label)]
pd_Φ = pd_Φ[[label for label in sentiment_list]]
pd_Φ = pd_Φ.round(4)
pd_Φ.to_parquet('./parquet_data/20250227_tweet_EM_sentiment.parquet')



# 変分ベイズアルゴリズムによる感情分析の結果を保存
pd_Φ = pd.read_parquet('./parquet_data/20250227_tweet_VB_Φ.parquet')
for label in sentiment_list:
	pd_Φ[label] = 0.0
# sentiment_listの分だけpd_Φの列が増えているが、zip関数の挙動的に要素数の少ないres_classに合わせて動作するため問題ない
for col, res in zip(pd_Φ.columns, res_class):
    # 各単語の感情分析結果を重み付けして感情分析結果を算出
	for label in sentiment_list:
		pd_Φ.loc[:, label] = pd_Φ.loc[:, label] + pd_Φ.loc[:, col] * res[sentiment_list.index(label)]
pd_Φ = pd_Φ[[label for label in sentiment_list]]
pd_Φ = pd_Φ.round(4)
pd_Φ.to_parquet('./parquet_data/20250227_tweet_VB_sentiment.parquet')



# ギブスサンプリングアルゴリズムによる感情分析の結果を保存
pd_Φ = pd.read_parquet('./parquet_data/20250227_tweet_CGS_Φ.parquet')
for label in sentiment_list:
	pd_Φ[label] = 0.0
# sentiment_listの分だけpd_Φの列が増えているが、zip関数の挙動的に要素数の少ないres_classに合わせて動作するため問題ない
for col, res in zip(pd_Φ.columns, res_class):
    # 各単語の感情分析結果を重み付けして感情分析結果を算出
	for label in sentiment_list:
		pd_Φ.loc[:, label] = pd_Φ.loc[:, label] + pd_Φ.loc[:, col] * res[sentiment_list.index(label)]
pd_Φ = pd_Φ[[label for label in sentiment_list]]
pd_Φ = pd_Φ.round(4)
pd_Φ.to_parquet('./parquet_data/20250227_tweet_CGS_sentiment.parquet')

In [7]:
df_tweet  = pd.read_parquet('./parquet_data/20250227_tweet_normalized.parquet')
documents = df_tweet['text'].tolist()

res_class  = []
res_probab = []
batch_size = 4
for idx in tqdm(range(0, len(documents), batch_size)):
	targets = documents[idx:idx+batch_size]
    
	inputs  = tokenizer(targets, return_tensors='pt', padding=True, truncation=True, max_length=512)
	inputs  = {key: value.to(device) for key, value in inputs.items()}
	outputs = model(**inputs)
	probab  = torch.softmax(outputs.logits, dim=1)
	results = [sentiment_list[idx] for idx in torch.argmax(probab, dim=1).tolist()]
	probab  = probab.tolist()
 
	# 0:negative, 1:neutral, 2:positive
	res_class.extend(results)
	res_probab.extend(probab)

# 各文書の分類結果の保存
df_tweet['classification'] = res_class
for idx, label in enumerate(sentiment_list):
    df_tweet[label] = [elem[idx] for elem in res_probab]

df_tweet.to_parquet('./parquet_data/20250227_tweet_normalized.parquet')

  0%|          | 0/511 [00:00<?, ?it/s]